<a href="https://colab.research.google.com/github/lavanyashukla/neural_networks/blob/master/Emotion%20Classifier%20-%20The%20Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotion Classifier - The Competition

# Welcome!
In this notebook we'll train an emotion classifier and deploy it to a tensorflow js frontend.  The first step is setting up the environment.

We’ll also set up Weights & Biases to log models metrics, inspect performance and share findings about the best architecture for the network. In this example we're using Google Colab as a convenient hosted environment, but you can run your own training scripts from anywhere and visualize metrics with W&B's experiment tracking tool.

### Running This Notebook
1. Click "Open in playground" to create a copy of this notebook for yourself.
2. Save a copy in Google Drive for yourself.
3. To enable a GPU, please click Edit > Notebook Settings. Change the "hardware accelerator" to GPU.
4. Step through each section below, pressing play on the code blocks to run the cells.

Results will be logged to a [shared W&B project page](https://app.wandb.ai/wandb/feb8-emotion).

We highly encourage you to fork this notebook, tweak the parameters, or try the model with your own dataset!

In [0]:
# Install wandb
!pip install -qq wandb
!pip install opencv-python
!pip install tensorflow

     |████████████████████████████████| 1.4MB 3.5MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 102kB 11.2MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 460kB 19.8MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 


In [0]:
#import libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import cv2
import subprocess
import os
import time
import wandb

## Load the fer2013 grayscale face emotion dataset

https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data

We manually do an 80/20 train-test split and cache the data to disk.

In [0]:
def load_fer2013(force=False):
    """Load the emotion dataset"""
    if force or not os.path.exists("fer2013"):
        print("Downloading the face emotion dataset...")
        subprocess.check_output(
            "curl -SL https://www.dropbox.com/s/opuvvdv3uligypx/fer2013.tar | tar xz", shell=True)
    print("Loading dataset...")
    if not os.path.exists('face_cache.npz'):
        data = pd.read_csv("fer2013/fer2013.csv")
        pixels = data['pixels'].tolist()
        width, height = 48, 48
        faces = []
        for pixel_sequence in pixels:
            pixs = pixel_sequence.split(' ')
            try:
                face = np.asarray(pixel_sequence.split(
                    ' '), dtype=np.uint8).reshape(width, height)
                face = cv2.resize(face.astype('uint8'), (width, height))
                faces.append(face.astype('float32'))
            except ValueError:
              print("Unable to load face.")

        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)
        emotions = pd.get_dummies(data['emotion']).as_matrix()

        val_faces = faces[int(len(faces) * 0.8):]
        val_emotions = emotions[int(len(faces) * 0.8):]
        train_faces = faces[:int(len(faces) * 0.8)]
        train_emotions = emotions[:int(len(faces) * 0.8)]
        np.savez('face_cache.npz', train_faces=train_faces, train_emotions=train_emotions,
                 val_faces=val_faces, val_emotions=val_emotions)
    cached = np.load('face_cache.npz')

    return cached['train_faces'], cached['train_emotions'], cached['val_faces'], cached['val_emotions']

# Deep Learning

We define a train() function with default inputs.  In the second cell we manually call training and convert the keras model into a tensorflow js model.

In [0]:
# Set default hyperparameters
default_config = {
    "learning_rate": 0.001,
    "batch_size": 32,
    "num_epochs": 10,
    "dropout": 0.2
}
def train():
  """Train an emotion classifier using wandb.config as input"""
  import tensorflow as tf
  import wandb
  tf.keras.backend.clear_session()
  # Inititialize W&B with default config options
  wandb.init(entity="wandb", project="feb8-emotion", config=default_config)
  config = wandb.config
  print(dict(config))
  
  # Load dataset
  input_shape = (48, 48, 1)
  train_faces, train_emotions, val_faces, val_emotions = load_fer2013()
  num_samples, num_classes = train_emotions.shape
  
  # Normalize data
  train_faces /= 255.
  val_faces /= 255.
  
  # Define the model
  optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
  #model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=input_shape, include_top=False)

  model = tf.keras.Sequential()
  '''
  # Set of Conv2D, Conv2D, MaxPooling2D layers with 32 and 64 filters
  model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu', input_shape = input_shape))
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', 
                    activation ='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(tf.keras.layers.Flatten())
  '''

  model.add(tf.keras.layers.Flatten(input_shape=input_shape))
  
  model.add(tf.keras.layers.Dense(128, activation="relu"))
  model.add(tf.keras.layers.Dropout(0.4))
  model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))
  model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                metrics=['accuracy'])

  # Save extra hyperparameter
  config.total_params = model.count_params()
    
  # Train the model
  model.fit(train_faces, train_emotions, batch_size=config.batch_size,
            epochs=config.num_epochs, verbose=1, callbacks=[
                wandb.keras.WandbCallback(data_type="image", labels=[
                              "Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"])
            ], validation_data=(val_faces, val_emotions))

  # Save the model locally
  model.save("emotion.h5")

In [0]:
# Train the model
train()

# Setup & serve the frontend

We're downloading and serving a pre-built React application from [github](https://github.com/vanpelt/emotion-detector)

In [0]:
# Download the frontend build
!rm -rf build
!wget -q https://github.com/vanpelt/emotion-detector/releases/download/stable/frontend.zip
!unzip -q frontend.zip

In [0]:
# Install tensorflowjs in a virtualenv
%pip install -q virtualenv
!virtualenv --no-site-packages venv && . venv/bin/activate && pip install -q tensorflowjs

In [0]:
# Quantize our trained model
!. venv/bin/activate && tensorflowjs_converter --input_format keras --quantization_bytes 2 emotion.h5 build/models

In [0]:
# Serve our custom UI
from subprocess import Popen
import portpicker
try:
  server.kill()
except NameError:
  pass
port = portpicker.pick_unused_port()
server = Popen(["cd ./build && python -m http.server %i" % port], shell=True,
               stdin=None, stdout=None, stderr=None, close_fds=True)

In [0]:
#Setup the interface for display
import IPython
html = open("./build/index.html").read()
body = html.replace('="/', '="https://localhost:{}/'.format(port),10)
body = body.replace("</head>", '<script type="text/javascript"/>window.BASE_URL = "https://localhost:{}/";google.colab.output.setIframeHeight(600)</script></head>'.format(port))
display(IPython.display.HTML(body))

# Hyper Parameter Sweeps

Full documentation [here](https://docs.wandb.com/library/sweeps/python-api).  

*WARNING* if you've run the train method in the main thread you must click "Restart runtime" before running a sweep.  Unfortunately tensorflow is only fork safe if it was never run in the main process.


In [0]:
# Configure the sweep
sweep_config = {
  "name": "Simple grid search",
  "method": "grid",
  "parameters": {
        "learning_rate": {
            "values": [0.001, 0.003, 0.005]
        },
        "batch_size": {
            "values": [32, 64, 128]
        },
        "dropout": {
            "values": [0.2, 0.3, 0.4]
        },
        "hidden_layer_size": {
            "values": [128, 256, 512]
        }
    }
}
wandb.reset_env()
sweep_id = wandb.sweep(sweep_config, project="feb8-emotion", entity="wandb")

In [0]:
# Run an agent with our training function
wandb.agent(sweep_id, function=train)

# Consuming sweep results

After the sweep has completed we can query for the best run and download it's weights.

In [0]:
api = wandb.Api()
sweep = api.sweep(f"wandb/emotion-oct29/{sweep_id}")
runs = sorted(sweep.runs, key=lambda run: run.summary.get("val_acc", 0), reverse=True)
val_acc = runs[0].summary.get("val_acc", 0)
print(f"Best run {runs[0].name} with {val_acc}% validation accuracy")
runs[0].file("model-best.h5").download(replace=True)
print("Best model saved to model-best.h5")